# KPIs & Scoring function

## KPIs

In [12]:
from collections import Counter
import string
import re
import numpy as np

### Muletillas

In [37]:
def calcular_muletillas(transcript, duracion=None, muletillas=None):
    if muletillas is None:
        muletillas = ["eh", "em", "este", "pues", "entonces", "bueno", "mmm", "o sea", "en plan"] 
        pausas = ["ppausaa"] # La pausa deberá ser detectada por el whisper/gpt de turno   

    vendedor_texto = " ".join(t["text"].lower() for t in transcript if t["speaker"] == "vendedor")

    # Eliminar signos de puntuación y tokenizar
    translator = str.maketrans('', '', string.punctuation)
    palabras_limpias = [word.translate(translator) for word in vendedor_texto.split()]

    # Contar muletillas y pausas
    muletillas_usadas = [word for word in palabras_limpias if word in muletillas]
    total_muletillas = len(muletillas_usadas)
    total_pausas = sum(1 for word in palabras_limpias if word in pausas)

    penalizacion = total_muletillas // 5 + total_pausas*10

    # Penalización extra si más del 70% de las muletillas son la misma
    if total_muletillas > 2:
        conteo = Counter(muletillas_usadas)
        muletilla_mas_frecuente, frecuencia = conteo.most_common(1)[0]
        if frecuencia / total_muletillas > 0.7:
            penalizacion += 10

    puntuacion = max(0, 100 - penalizacion)  # nunca bajar de 0

    return  {
        "puntuacion": puntuacion,
        "penalizacion": penalizacion,
        "total_muletillas": total_muletillas,
        "total_pausas": total_pausas,
        "muletillas_usadas": muletillas_usadas
    }

### Claridad y complejidad

In [7]:
def calcular_claridad(transcript):
    # Lista de tecnicismos a detectar
    tecnicismos = [
        "kpi", "roi", "funnel", "pipeline", "crm", "saas", "onboarding", "cloud computing", "escalabilidad", 
        "benchmarking", "sla", "bant", "mql", "sql", "tco", "erp", "seo", "api", "throughput", 
        "lead nurturing", "proof of concept", "poc", "framework"
    ]
    
    # Expresiones de lenguaje positivo
    lenguaje_positivo = [
        "absolutamente", "excelente pregunta", "me encanta esa idea", "claro que sí", "con gusto", 
        "es un placer", "podemos lograr", "le aseguro", "le confirmo", "fantástico", "maravilloso", 
        "por supuesto", "genial", "tiene razón", "lo valoro mucho", "vamos a avanzar", 
        "estoy seguro de que", "lo conseguiremos", "gran punto"
    ]
    
    # Unir todo el texto del vendedor
    vendedor_texto = " ".join(t["text"].lower() for t in transcript if t["speaker"] == "vendedor")

    # Eliminar signos de puntuación para facilitar análisis
    translator = str.maketrans('', '', string.punctuation + "¿¡")
    texto_limpio = vendedor_texto.translate(translator)

    # Dividir en frases por punto, ? o !
    frases = [f.strip() for f in texto_limpio.replace("?", ".").replace("!", ".").split(".") if f.strip()]

    # Inicializar métricas
    penalizacion = 0
    bonificacion = 0
    
    # ---- Penalización por tecnicismos ----
    for term in tecnicismos:
        count = vendedor_texto.count(term)
        penalizacion += 15 * count

    # ---- Penalización por frases largas ----
    for frase in frases:
        palabras = frase.split()
        if len(palabras) > 25:
            penalizacion += 10

    # ---- Bonificación por lenguaje positivo ----
    for expresion in lenguaje_positivo:
        if expresion in vendedor_texto:
            bonificacion += 10
            break  # solo bonificamos una vez aunque haya varias

    # ---- Calcular puntuación final ----
    puntuacion = max(0, min(100, 100 - penalizacion + bonificacion))

    return {
        "puntuacion": puntuacion,
        "penalizacion": penalizacion,
        "bonificacion": bonificacion,
        "num_frases_largas": sum(1 for f in frases if len(f.split()) > 25)
    }

### Participación y dinámica

In [8]:
def calcular_participacion_dinamica(transcript):
    # Heurísticas de interrupciones (puedes ampliar con más expresiones típicas)
    patrones_interrupcion = ["sí", "claro", "vale", "déjame", "espera", "pero"]
    
    # Heurísticas de escucha activa
    patrones_escucha_activa = [
        "como me dijiste", "entiendo que", "lo que comentabas", "si no me equivoco", 
        "resumiendo", "entonces lo que", "si entiendo bien", "según lo que me dices"
    ]
    
    # ---- Calcular palabras totales por speaker ----
    palabras_vendedor = 0
    palabras_cliente = 0
    interrupciones = 0
    escucha_activa = False
    
    for i, turno in enumerate(transcript):
        texto = turno["text"].lower()
        palabras = len(texto.split())
        
        if turno["speaker"] == "vendedor":
            palabras_vendedor += palabras
            
            # Buscar escucha activa
            if any(pat in texto for pat in patrones_escucha_activa):
                escucha_activa = True
                
            # Detectar interrupciones: vendedor habla justo después del cliente con patrón
            if i > 0 and transcript[i-1]["speaker"] == "cliente":
                if any(texto.startswith(pat) for pat in patrones_interrupcion):
                    interrupciones += 1
                    
        elif turno["speaker"] == "cliente":
            palabras_cliente += palabras
    
    total_palabras = palabras_vendedor + palabras_cliente
    
    # ---- Penalizaciones y bonificaciones ----
    penalizacion = 0
    bonificacion = 0
    
    # Interrupciones
    penalizacion += interrupciones * 20
    
    # Monólogo: más del 70% de las palabras
    if total_palabras > 0 and (palabras_vendedor / total_palabras) > 0.7:
        penalizacion += 25
    
    # Escucha activa
    if escucha_activa:
        bonificacion += 15
    
    # ---- Puntuación final ----
    puntuacion = max(0, min(100, 100 - penalizacion + bonificacion))
    
    return {
        "puntuacion": puntuacion,
        "penalizacion": penalizacion,
        "bonificacion": bonificacion,
        "interrupciones": interrupciones,
        "monologo": (palabras_vendedor / total_palabras > 0.7 if total_palabras > 0 else False),
        "escucha_activa": escucha_activa
    }

### Cobertura de temas y palabras clave

In [9]:
def calcular_cobertura_temas(transcript, temas_clave=None):
    # Temas clave esperados. Aquí habrá que hacer un trabajo de customización 
    temas_clave = {
        "siguiente paso": ["siguiente paso", "next step", "seguimiento"],
        "propuesta de valor": ["propuesta de valor", "beneficio", "ventaja competitiva"],
        "precio": ["precio", "coste", "inversión", "tarifa"]
    }
    
    # Palabras clave de "next step"
    palabras_next_step = [
        "agendemos", "enviar", "revisaremos", "próximo", "¿cuándo?", "¿qué día?", "confirmar",
        "¿de acuerdo?", "siguientes pasos", "coordinemos", "pongamos", "paso a seguir", 
        "formalizar", "¿cuál es el mejor momento?", "le parece si", "¿puede usted?", "¿a qué hora?"
    ]
    
    vendedor_texto = " ".join(t["text"].lower() for t in transcript if t["speaker"] == "vendedor")
    cliente_texto  = " ".join(t["text"].lower() for t in transcript if t["speaker"] == "cliente")

    penalizacion = 0
    bonificacion = 0
    
    # ---- Temas clave olvidados ----
    temas_olvidados = []
    for tema, keywords in temas_clave.items():
        if not any(k in vendedor_texto for k in keywords):
            penalizacion += 15
            temas_olvidados.append(tema)
    
    # ---- Objeciones críticas no resueltas ----
    objeciones_cliente = ["precio", "coste", "caro", "elevado", "demasiado"]
    objecion_detectada = any(word in cliente_texto for word in objeciones_cliente)
    
    if objecion_detectada:
        # ¿Responde el vendedor?
        respuesta = any(word in vendedor_texto for word in ["precio", "coste", "inversión", "valor"])
        if not respuesta:
            penalizacion += 30
    
    # ---- Definición de próximo paso ----
    if any(exp in vendedor_texto for exp in palabras_next_step):
        bonificacion += 15
    
    # ---- Puntuación final ----
    puntuacion = max(0, min(100, 100 - penalizacion + bonificacion))
    
    return {
        "puntuacion": puntuacion,
        "penalizacion": penalizacion,
        "bonificacion": bonificacion,
        "temas_olvidados": temas_olvidados,
        "objecion_no_resuelta": objecion_detectada and not respuesta,
        "next_step_definido": bonificacion > 0
    }

### Indice de preguntas

In [53]:
def calcular_indice_preguntas(transcript):
    # Definiciones de sets de preguntas
    preguntas_cerradas = [
        "¿necesita una demostración?", "¿le interesa el producto?", "¿está de acuerdo con el precio?", 
        "¿tiene alguna pregunta?", "¿me escucha bien?", "¿terminamos aquí?", "¿lo ve factible?", 
        "¿quiere seguir adelante?", "¿tiene x presupuesto?", "¿le parece bien esto?", 
        "¿podemos empezar mañana?", "¿sí o no?", "¿cuenta con la autoridad?"
    ]
    
    preguntas_sondeo = [
        "¿cómo afecta este problema a su rentabilidad?", "¿cuál es el costo de no resolver esto a tiempo?", 
        "¿qué prioridades tiene para el próximo trimestre?", "¿qué criterios usará para tomar la decisión?", 
        "¿qué alternativas ha evaluado hasta ahora?", "¿qué impacto tendría en su equipo?", 
        "¿qué pasaría si mantiene el status quo?", "¿cómo mide el éxito de este proyecto?", 
        "¿quién más está involucrado en la decisión?", "¿qué le preocupa más de la solución?", 
        "¿por qué es importante esto ahora?"
    ]
    
    preguntas_irrelevantes = [
        "¿hizo buen tiempo hoy?", "¿cómo va el tráfico por su zona?", "¿qué tal su fin de semana?", 
        "¿leyó las noticias de hoy?", "¿qué opina de", "¿tiene planes para las fiestas?", 
        "¿qué equipo de fútbol le gusta?", "¿dónde está ubicado su edificio?", "¿viajó mucho para llegar hoy?"
    ]
    
    # Solo texto del vendedor
    vendedor_texto = " ".join(t["text"].lower() for t in transcript if t["speaker"] == "vendedor")
    
    # Extraer todas las preguntas que hace el vendedor
    preguntas = re.findall(r"¿.*?\?", vendedor_texto, re.DOTALL)
    
    total_preguntas = len(preguntas)
    count_cerradas = 0
    count_sondeo = 0
    count_irrelevantes = 0
    
    for p in preguntas:
        p = p.strip()
        if any(p.startswith(pc) for pc in preguntas_cerradas):
            count_cerradas += 1
        elif any(p.startswith(ps) for ps in preguntas_sondeo):
            count_sondeo += 1
        elif any(p.startswith(pi) for pi in preguntas_irrelevantes):
            count_irrelevantes += 1
    
    penalizacion = 0
    bonificacion = 0
    
    # Penalización si cerradas > 60% del total
    if total_preguntas > 0 and (count_cerradas / total_preguntas) > 0.6:
        penalizacion += 15
    
    # Penalización por irrelevantes
    penalizacion += count_irrelevantes * 10
    
    # Bonificación por sondeo
    bonificacion += count_sondeo * 10
    
    # Score final
    puntuacion = max(0, min(100, 100 - penalizacion + bonificacion))
    
    return {
        "puntuacion": puntuacion,
        "total_preguntas": total_preguntas,
        "preguntas": preguntas,
        "cerradas": count_cerradas,
        "sondeo": count_sondeo,
        "irrelevantes": count_irrelevantes,
        "penalizacion": penalizacion,
        "bonificacion": bonificacion
    }

### PPM y variabilidad

In [47]:
def calcular_ppm_variabilidad(transcript):
    # Consideramos solo al vendedor
    turnos_vendedor = [t for t in transcript if t["speaker"] == "vendedor"]
    
    if not turnos_vendedor:
        return {
            "puntuacion": 100,
            "ppms": [],
            "media_ppm": None,
            "variabilidad": None,
            "penalizacion": 0,
            "bonificacion": 0
        }
    
    ppms = []
    total_palabras = 0
    total_duracion = 0.0
    
    for t in turnos_vendedor:
        palabras = len(t["text"].split())
        duracion = t["duracion"]
        ppm = palabras / (duracion / 60)
        
        ppms.append(ppm)
        total_palabras += palabras
        total_duracion += duracion
    
    media_ppm = total_palabras / (total_duracion / 60)
    variabilidad = np.std(ppms) if len(ppms) > 1 else 0
    
    penalizacion = 0
    bonificacion = 0
    
    # Penalización por velocidad extrema
    if media_ppm < 120 or media_ppm > 170:
        penalizacion += 10
    
    # Penalización por variabilidad extrema
    if variabilidad > 30:  # umbral configurable
        penalizacion += 15
    
    # Bonificación por ritmo ideal
    if 130 <= media_ppm <= 150:
        bonificacion += 10
    
    puntuacion = max(0, min(100, 100 - penalizacion + bonificacion))
    
    return {
        "puntuacion": puntuacion,
        "ppms": ppms,
        "media_ppm": round(media_ppm, 1),
        "variabilidad": round(variabilidad, 1),
        "penalizacion": penalizacion,
        "bonificacion": bonificacion
    }

## Scoring function

In [42]:
def evaluar_transcript_global(transcript):
    # Factores de ponderación
    pesos = {
        "muletillas_pausas": 0.15,
        "claridad": 0.15,
        "participacion": 0.25,
        "cobertura": 0.15,
        "preguntas": 0.20,
        "ppm": 0.10
    }
    
    # Evaluaciones individuales
    res_muletillas = calcular_muletillas(transcript)
    res_claridad = calcular_claridad(transcript)
    res_participacion = calcular_participacion_dinamica(transcript)
    res_cobertura = calcular_cobertura_temas(transcript)
    res_preguntas = calcular_indice_preguntas(transcript)
    res_ppm = calcular_ppm_variabilidad(transcript)
    
    # Extraer puntuaciones
    scores = {
        "muletillas_pausas": res_muletillas["puntuacion"],
        "claridad": res_claridad["puntuacion"],
        "participacion": res_participacion["puntuacion"],
        "cobertura": res_cobertura["puntuacion"],
        "preguntas": res_preguntas["puntuacion"],
        "ppm": res_ppm["puntuacion"]
    }
    
    # Calcular puntuación ponderada global
    puntuacion_final = sum(scores[k] * pesos[k] for k in scores)
    
    return {
        "puntuacion_global": round(puntuacion_final, 1),
        "detalle": scores,
        "pesos": pesos
    }


## Testing examples!

### Transcripts 1-10 (Text samples from Conversa WorkDoc)

In [50]:
transcript_1 = [
    {"speaker": "vendedor", "text": "Buenos días, soy Elena de Innovaciones Muestrales. Soy consciente de lo valioso que es su tiempo, así que permítame ir directamente al punto. Trabajamos con laboratorios de alto volumen como el suyo. ¿Le interesa saber cómo hemos ayudado a otros clientes a reducir sus errores de muestreo, la hemólisis y, de manera crucial, el tiempo total de procesamiento en un 15%?"},
    {"speaker": "cliente", "text": "Buenos días. Esos son indicadores que siempre estamos buscando mejorar. La eficiencia y la calidad de las muestras son vitales. Cuénteme, ¿qué tienen exactamente para ofrecer que no tengamos ya?"},
    {"speaker": "vendedor", "text": "Excelente pregunta. Nuestra línea de jeringuillas utiliza un diseño de doble pared que reduce la turbulencia de la muestra en un 40% durante la extracción, impactando directamente en la estabilidad. Pero antes de abrumarla con datos técnicos, necesito comprender mejor sus desafíos: ¿Qué prioridad tiene para su laboratorio el tiempo de procesamiento en el proceso de muestras críticas y cuál es el costo promedio (incluyendo personal, reactivos y tiempo de cama) que asumen por un error de reprocesamiento al día?"},
    {"speaker": "cliente", "text": "El tiempo es una prioridad media-alta, pues siempre nos presiona el área clínica en horario nocturno. Un reprocesamiento nos cuesta cerca de 50€, sin contar la frustración del personal. Nuestra tasa de error actual es, desafortunadamente, del 3.5%, y eso es lo que más nos gustaría reducir."},
    {"speaker": "vendedor", "text": "Lo valoro mucho. Entonces, si entendí correctamente, reducir esos costos de reprocesamiento (esos 50€ diarios) y bajar esa tasa de error del 3.5% son los indicadores clave que justificarían explorar un nuevo proveedor, ¿verdad? Y, para asegurar una decisión ágil, ¿quién más en su equipo estaría involucrado en evaluar y aprobar la prueba técnica, además de usted?"},
    {"speaker": "cliente", "text": "Es correcto. Si me demuestran que bajan la tasa de error al menos a la mitad, tienen mi atención. El equipo de Jefatura de Laboratorio y yo tomamos la decisión técnica. Pero, honestamente, el precio por unidad sigue siendo mi gran preocupación. Ya tenemos el presupuesto cerrado para este trimestre y cualquier sobrecoste es un problema operativo."},
    {"speaker": "vendedor", "text": "Claro que sí, es una preocupación muy válida. Permítame abordar eso de inmediato. Nuestro precio unitario es un 8% superior al de su proveedor actual. Sin embargo, nuestros clientes en su segmento reportan una reducción media de 50% en la tasa de error en el primer trimestre. En su caso, eso se traduce en un ahorro potencial de 15€ a 20€ por día en evitar el reprocesamiento. ¿Podría confirmar si un ahorro neto de 300€ a 400€ al mes justificaría la diferencia de precio unitario?"},
    {"speaker": "cliente", "text": "La cifra de ahorro es interesante y esa justificación tiene peso. Aún así, un 8% es un cambio en el gasto operativo. ¿Hay algún tipo de acuerdo de volumen o escalonado que podamos negociar una vez que se pruebe la reducción del error y veamos esos KPIs de ahorro reflejados?"},
    {"speaker": "vendedor", "text": "Sí, tenemos un modelo de incentivos por volumen que podemos revisar post-prueba. Veo que estamos alineadas en el potencial de ahorro. Volviendo a la parte técnica, usted mencionó que el tiempo de procesamiento es una prioridad media-alta. ¿Qué tan crítico es el tiempo de centrifugado en su pico nocturno, y cuánto han intentado reducirlo antes sin éxito?"},
    {"speaker": "cliente", "text": "Es crítico en la noche, con personal reducido. Hemos intentado cambiar las centrifugadoras, pero el costo de capital fue demasiado alto. Necesitamos una solución que no implique una inversión inicial grande en maquinaria, algo que cambie el proceso, no el equipo."},
    {"speaker": "vendedor", "text": "Comprendido. Su preocupación no es solo la calidad, sino la eficiencia bajo presión. ¿Y qué pasa con la satisfacción del personal? ¿El equipo de flebotomía ha reportado alguna frustración específica con el sistema de extracción actual que le preocupe en términos de rotación?"},
    {"speaker": "cliente", "text": "Sí, el personal a menudo se queja de la dificultad en la alineación con nuestro sistema actual, lo que a veces requiere un segundo intento. Eso ralentiza todo y aumenta la frustración. Honestamente, es un factor de desgaste."},
    {"speaker": "vendedor", "text": "Gracias por compartir eso. Y es aquí donde nuestra jeringuilla ofrece un triple valor. El diseño ergonómico reduce la dificultad de alineación, disminuyendo los segundos intentos. El diseño interno minimiza la turbulencia, permitiendo reducir el tiempo de centrifugado en un 15%. Me encanta esa idea de ofrecerle una solución integral: menor error, menor tiempo de procesamiento y mejor satisfacción del personal sin inversión de capital."},
    {"speaker": "vendedor", "text": "¿Cuál es el próximo paso para que usted y la Jefatura de Laboratorio puedan validar este triple beneficio: menor error, menor tiempo y mejor ergonomía?"},
    {"speaker": "cliente", "text": "Necesito la información técnica detallada y el informe de un par de estudios de caso con cifras de satisfacción. Luego, sí, una prueba piloto. Pero necesito el kit de prueba lo antes posible para empezar a medir."},
    {"speaker": "vendedor", "text": "Perfecto. Para validar el ahorro y la eficiencia, agendemos un encuentro de 15 minutos el próximo martes con la Jefatura. Le enviaremos inmediatamente el estudio de caso, los datos técnicos y coordinaremos el envío de su kit de prueba piloto de dos semanas. ¿Martes a las 10:00 am le funciona para formalizar el piloto y revisar el kit?"},
    {"speaker": "cliente", "text": "El martes a las 10:00 am está bien. Agendemos."}
]

transcript_2 = [
    {"speaker": "vendedor", "text": "Hola, ¿eh? Soy Ricardo. Mira, mi empresa ayuda a reducir el error de hemólisis en un 60%. Digamos, que básicamente vendemos la jeringuilla XZ, que tiene throughput optimizado. ¿Cuánto le cuesta al laboratorio un error de reprocesamiento en términos de tiempo y recursos?"},
    {"speaker": "cliente", "text": "Es una buena pregunta. El reprocesamiento nos cuesta tiempo y reactivos, calculo unos 50€ por caso. Pero mi problema principal ahora es la falta de stock de nuestro proveedor actual. Necesito una solución que sea confiable logísticamente."},
    {"speaker": "vendedor", "text": "O sea, entiendo que la logística es una prioridad, ¿verdad? Pero, a ver, si pudiéramos eliminar el 60% de sus reprocesamientos (lo que le ahorraría más de 1,000€ al mes), ¿quitaría eso la urgencia de la logística por un momento?"},
    {"speaker": "cliente", "text": "No, no la quitaría. Necesito ambos. Si su proveedor actual no tiene stock, no me sirve de nada el ahorro. Y, además, nuestro jefe de laboratorio solo acepta productos con la certificación ISO-SaaS."},
    {"speaker": "vendedor", "text": "Mire, la certificación ISO-SaaS está en proceso, la tendremos en el próximo trimestre. Este, por ahora, pues, la calidad de la jeringuilla es superior. ¿Podría usted decidir una prueba piloto solo con el ahorro del 60% de la hemólisis?......."},
    {"speaker": "cliente", "text": "Definitivamente no. Sin la certificación requerida por la Jefatura de Laboratorio, no puedo avanzar. Y la logística es un tema de inventario diario. No me está ofreciendo garantía de suministro."},
    {"speaker": "vendedor", "text": "Bueno, la verdad, es que le estamos ofreciendo el mejor producto del mercado. ¿Entonces, agendamos una reunión para revisar el business case en dos semanas, o no? Debería aprovechar esta oportunidad."},
    {"speaker": "cliente", "text": "No, Ricardo. Le agradezco la llamada, pero si no tienen la certificación ni garantizan la logística, no tiene sentido avanzar. Adiós."}
]

transcript_3 = [
    {"speaker": "vendedor", "text": "Hola, ¿eh? Soy Paula de Servicios Quirúrgicos. Mira, mi objetivo es ahorrarle dinero. Digamos, que hemos visto que su laboratorio tiene una tasa de error del 3.5%. Este, ¿cuál es el costo total aproximado que le generan al mes los reprocesamientos de muestras y el costo de oportunidad de ese tiempo perdido?"},
    {"speaker": "cliente", "text": "Buenos días, Paula. Es una cifra difícil de estimar en caliente, pero sé que el reprocesamiento nos cuesta cerca de 50€ por caso. El costo de oportunidad es lo que más me duele, porque nos retrasa la implementación de un nuevo sistema de trazabilidad."},
    {"speaker": "vendedor", "text": "Excelente. O sea, entiendo que el retraso en el sistema de trazabilidad es su principal dolor ahora mismo, ¿verdad? A ver, ¿cuánto tiempo de personal se pierde a la semana en ese proceso y qué proyectos estratégicos se están retrasando por dedicarle ese tiempo?"},
    {"speaker": "cliente", "text": "Diría que perdemos unas 15 a 20 horas a la semana, sobre todo en las noches. Si resolviéramos esto, liberaríamos a dos técnicos para el proyecto de trazabilidad. Honestamente, si nuestra jeringuilla no es compatible con el sistema de etiquetado automático actual, la conversación termina aquí."},
    {"speaker": "vendedor", "text": "Comprendido. Pues, sí, la compatibilidad con el sistema de etiquetado está certificada. Mira, para volver al tema del presupuesto, que sé que es apretado, el precio por unidad es un 15% más alto. ¿Cómo justifica una compra que excede el presupuesto trimestral, a pesar del ahorro de esas 20 horas/semana y la liberación del proyecto de trazabilidad?"},
    {"speaker": "cliente", "text": "El CFO. La justificación de las horas ahorradas tiene que ser muy sólida. Y tenemos otro gran problema: la logística. Nuestro proveedor actual tiene interrupciones constantes y no tenemos stock de reserva. ¿Ustedes garantizan un SLA de suministro del 99.9%?"},
    {"speaker": "vendedor", "text": "Claro que sí. Este, la verdad es que garantizamos el SLA con un sistema de reserva automatizado. Digamos, que eso se resuelve. Pero, volvamos a lo importante: El CFO. Si le muestro un informe que proyecta que nuestra jeringuilla le permitirá finalizar el proyecto de trazabilidad dos meses antes, ¿aprobaría él el 15% extra?"},
    {"speaker": "cliente", "text": "Si logran demostrar que liberan recursos para ese proyecto, sí, es muy probable. Es la prioridad número uno de la dirección."},
    {"speaker": "vendedor", "text": "Fantástico. Mira, eso me parece un excelente próximo paso. Le enviaré un Business Case centrado en el costo de oportunidad del retraso de la trazabilidad. Luego, agendemos la reunión de 30 minutos el próximo jueves, e invitamos al CFO, para que usted pueda formalizar la prueba piloto con la información completa. ¿Jueves a las 10:00 am le funciona?"},
    {"speaker": "cliente", "text": "Jueves a las 10:00 am. Perfecto, agendemos."}
]

transcript_4 = [
    {"speaker": "vendedor", "text": "Buenos días. Soy Rafael de Biosoluciones. Hemos identificado su laboratorio como un líder en volumen de procesamiento. Mi objetivo es claro: reducir sus errores de muestreo. ¿Cuál es el costo promedio que le genera un error de reprocesamiento en términos de tiempo de personal y uso de reactivos por mes?"},
    {"speaker": "cliente", "text": "Buenos días. Es una buena pregunta. Calculo que, por nuestra tasa de error actual del 3%, perdemos unos 1,200€ al mes entre tiempo, reactivos y costo de envío de muestras. Es una cifra importante para mí."},
    {"speaker": "vendedor", "text": "Comprendido. Excelente dato. Nuestro modelo de jeringuilla, la Bio-Estable, está diseñada con un sistema de doble cámara que maneja una presión iso-osmótica en el émbolo de 1.05 milibares. Esto es crucial porque minimiza el cizallamiento celular. Además, el material es un polímero de etileno-vinil-acetato inerte, lo que elimina la necesidad de preacondicionamiento. Esta innovación está respaldada por nuestra ISO-9001 y un SLA de rendimiento del 99.8%. Nuestro protocolo lean garantiza la trazabilidad del lote desde la fabricación hasta su uso final. ¿No le parece que la garantía de throughput que ofrecemos es superior?"},
    {"speaker": "cliente", "text": "Demasiada información técnica de golpe. El dato del ahorro es bueno, pero ¿pueden garantizarnos un suministro constante? Nuestro proveedor actual tiene serios problemas de logística y eso afecta nuestro inventario diario. La logística es mi prioridad número uno."},
    {"speaker": "vendedor", "text": "Sí, la logística es importante, pero permítame enfatizar el valor técnico. La Bio-Estable también tiene un recubrimiento de sílice amorfa en la superficie interior que mejora el índice de refrangibilidad de la muestra, optimizando la lectura espectrofotométrica en un 12%. Estamos hablando de ciencia de vanguardia que supera el estándar de la industria. ¿Podría usted confirmar que el ahorro en reprocesamiento es suficiente para compensar un 10% de aumento en el precio unitario?"},
    {"speaker": "cliente", "text": "Estoy de acuerdo con el ahorro, pero si no tengo la garantía de que voy a tener el producto a tiempo, el ahorro no existe. Y ese 10% de aumento de precio debe ser justificado ante el CFO. ¿Quién más en su equipo puede darme la garantía logística con cifras concretas?"},
    {"speaker": "vendedor", "text": "Tiene razón, la logística y el CFO. Le confirmo que tenemos una reserva de seguridad del 20% para todos nuestros clientes clave. Respecto al CFO: nuestra certificación Six Sigma en control de calidad justifica el precio. Para avanzar, agendemos una reunión el martes para que le envíe el informe técnico detallado. ¿Martes a las 10:00 am le funciona?"},
    {"speaker": "cliente", "text": "Necesito la garantía de suministro por escrito y necesito que el CFO hable con alguien de finanzas. No veo el valor en una reunión puramente técnica ahora mismo. Gracias."}
]

transcript_5 = [
    {"speaker": "vendedor", "text": "Buenos días. Soy Matías de Soluciones Médicas. Mi objetivo es claro: reducir su tasa de error de muestreo y el tiempo de respuesta en urgencias. ¿Cuál es el principal desafío que le está impidiendo alcanzar una tasa de error inferior al 1.5% actualmente?"},
    {"speaker": "cliente", "text": "Buenos días. El desafío es doble: primero, la ergonomía de nuestro kit actual frustra al personal, lo que lleva a errores; y segundo, el precio. Cualquier nuevo producto que consideremos tiene que ser competitivo en costes."},
    {"speaker": "vendedor", "text": "Comprendido. Excelente enfoque en la ergonomía. Hablemos de eso. La frustración del personal por la ergonomía, ¿ha provocado un aumento en la rotación o en las bajas por estrés en el último año?"},
    {"speaker": "cliente", "text": "Sí. Hemos tenido un 15% de rotación en flebotomía. Es muy costoso. Pero, insisto: si son más caros que nuestro proveedor actual, es un obstáculo insalvable para mí."},
    {"speaker": "vendedor", "text": "Entiendo que el precio es vital. Le aseguro que el diseño de nuestra jeringuilla resuelve el problema de la fatiga muscular gracias a su agarre de liberación suave. Esto directamente reduce la insatisfacción que usted mencionó. ¿Podría compartir conmigo qué otras áreas de su laboratorio se benefician más de la reducción en el tiempo de procesamiento de muestras críticas?"},
    {"speaker": "cliente", "text": "El área de diagnóstico. Si las muestras tardan menos en procesarse, los resultados llegan más rápido al médico. Pero, volviendo al precio, ¿es usted consciente de que estamos sujetos a un contrato de precios fijos por volumen con nuestro proveedor actual?"},
    {"speaker": "vendedor", "text": "Sí, lo entiendo. Y es muy importante. La reducción en el tiempo de procesamiento de muestras sensibles es un gran valor. Nuestra aguja ultrafina reduce el tiempo en la camilla en un 20%. ¿Le parece más urgente resolver el problema de la rotación o el problema del tiempo en urgencias?"},
    {"speaker": "cliente", "text": "Ambos, pero el coste lo define todo. Si no me da una cifra ahora mismo, o una banda de precios, no puedo justificar el tiempo de una reunión de demostración."},
    {"speaker": "vendedor", "text": "Tiene razón en que necesita la información financiera. Le enviaré un white paper que desglosa el costo total de la fatiga del personal. Agendemos un breve follow-up para discutir ese informe la próxima semana, ¿de acuerdo?"},
    {"speaker": "cliente", "text": "Un white paper no es el precio. Y, además, necesito la seguridad de que su empresa tiene la capacidad logística para abastecernos sin fallos. Nuestro proveedor tiene problemas de stock serios."},
    {"speaker": "vendedor", "text": "Claro que sí. Le confirmo que nuestra logística es robusta. De hecho, usamos un ERP de planificación avanzado que elimina los problemas de stock. ¿Necesita que le envíe también nuestra certificación de capacidad de almacenamiento para que tenga total tranquilidad logística?"},
    {"speaker": "cliente", "text": "Sí, necesito eso. Pero sobre el precio... ¿estamos hablando de un 5% o un 50% de diferencia con mi precio actual? Necesito un rango antes de agendar."},
    {"speaker": "vendedor", "text": "Entiendo la urgencia. Le prometo que el precio es competitivo. Agendemos la reunión de 20 minutos el próximo miércoles, invitamos al Jefe de Logística y le enviaremos el rango de precios en ese mismo email. ¿Miércoles a las 3:00 pm le funciona para formalizar el envío de la información?"},
    {"speaker": "cliente", "text": "No, Matías. No puedo agendar una reunión para que me envíen información que debí recibir en esta llamada. Le enviaré un correo con el precio máximo que podemos aceptar. Gracias."}
]

transcript_6 = [
    {"speaker": "vendedor", "text": "Buenos días. Soy Ana de Muestras Seguras. Eh, le llamo porque hemos visto que su laboratorio es un referente. Mira, mi objetivo es que vean cómo digamos que podemos reducir sus errores de muestreo. ¿Cuál es el principal desafío que le impide alcanzar una tasa de error inferior al 1.5% actualmente?"},
    {"speaker": "cliente", "text": "Buenos días. El desafío es doble: primero, la ergonomía de nuestro kit actual frustra al personal; y, lo más importante, el precio. Cualquier nuevo producto que consideremos tiene que ser competitivo en costes."},
    {"speaker": "vendedor", "text": "Comprendido. O sea, entiendo que el coste y la ergonomía son las dos variables clave. A ver, hablemos de la ergonomía, que es donde nuestro diseño destaca. La frustración del personal, ¿ha provocado un aumento en la rotación o en las bajas por estrés en el último año?"},
    {"speaker": "cliente", "text": "Sí. Hemos tenido un 15% de rotación en flebotomía. Es muy costoso. Pero, insisto: si son más caros que nuestro proveedor actual, es un obstáculo insalvable para mí."},
    {"speaker": "vendedor", "text": "Sí, lo entiendo. Bueno, le aseguro que nuestro diseño ergonómico reduce la fatiga. Esto directamente resuelve la insatisfacción que usted mencionó. ¿Podría compartir conmigo qué otras áreas de su laboratorio se benefician más de la reducción en el tiempo de procesamiento de muestras críticas?"},
    {"speaker": "cliente", "text": "El área de diagnóstico y el servicio de urgencias. Si las muestras tardan menos en procesarse, los resultados llegan más rápido. Pero, volviendo al precio, ¿es usted consciente de que estamos sujetos a un contrato de precios fijos por volumen con nuestro proveedor actual?"},
    {"speaker": "vendedor", "text": "Claro que sí. Lo importante es que la reducción en el tiempo de procesamiento de muestras sensibles es un gran valor para urgencias. Nuestra aguja agiliza el proceso. ¿Le parece más urgente resolver el problema de la rotación o el problema del tiempo en urgencias?"},
    {"speaker": "cliente", "text": "Ambos, pero el coste lo define todo. Necesito una cifra ahora mismo, o una banda de precios, para saber si vale la pena hablar con urgencias. Si no me da una cifra, no puedo avanzar."},
    {"speaker": "vendedor", "text": "Tiene razón en que necesita la información financiera. Le enviaré un informe que desglosa el ahorro total que logran nuestros clientes. Agendemos un breve follow-up para discutir ese informe la próxima semana, ¿de acuerdo? (PPAUSAA: 4 segundos)"},
    {"speaker": "cliente", "text": "Un informe de ahorro no es el precio. Y además, la logística. Necesito la seguridad de que su empresa tiene la capacidad logística para abastecernos sin fallos. Nuestro proveedor tiene problemas de stock serios."},
    {"speaker": "vendedor", "text": "Claro que sí. Le confirmo que nuestra logística es robusta. Este, de hecho, usamos un sistema de ERP avanzado que elimina los problemas de stock. ¿Necesita que le enviemos también nuestra certificación de capacidad de almacenamiento para que tenga total tranquilidad logística?"},
    {"speaker": "cliente", "text": "Sí, necesito eso. Pero sobre el precio... ¿estamos hablando de un 5% o un 50% de diferencia con mi precio actual? Necesito un rango claro."},
    {"speaker": "vendedor", "text": "Entiendo la urgencia. Le prometo que el precio es competitivo. Agendemos la reunión de 20 minutos el próximo miércoles, invitamos al Jefe de Logística, y le enviaremos el rango de precios en ese mismo email. ¿Miércoles a las 3:00 pm le funciona para formalizar el envío de la información?"},
    {"speaker": "cliente", "text": "No, Ana. No puedo agendar una reunión para que me envíen información que debí recibir en esta llamada. Le enviaré un correo con el precio máximo que podemos aceptar. Gracias."}
]

transcript_7 = [
    {"speaker": "vendedor", "text": "Buenos días. Soy Rodrigo de Bio-Excelencia. Mi enfoque es la eficiencia. Hemos visto que su laboratorio busca optimizar el tiempo de respuesta en urgencias. ¿Qué tan crítico es el tiempo de respuesta en sus muestras más sensibles y cómo mediría usted el éxito de una nueva herramienta de extracción?"},
    {"speaker": "cliente", "text": "Buenos días. Es una prioridad. El éxito se mediría en dos puntos: la reducción de nuestra tasa de hemólisis (actualmente 3.5%) y la minimización del esfuerzo muscular de nuestro personal de flebotomía. Y debo ser franca: el precio lo define todo."},
    {"speaker": "vendedor", "text": "Comprendido. (PPAUSAA: 4 segundos) Excelente que se enfoque en el factor humano. Para priorizar: Hablemos del riesgo. La frustración del personal, ¿ha provocado un aumento en la rotación o en las bajas por estrés en el último año? Necesito una cifra de costo de reemplazo."},
    {"speaker": "cliente", "text": "Sí, tenemos un 15% de rotación en flebotomía, y cada reemplazo nos cuesta unos 2,000€. Es un dolor constante. El precio es la clave, Rodrigo. Si son más caros, es un obstáculo insalvable."},
    {"speaker": "vendedor", "text": "Sí, entiendo el precio. Le aseguro que el diseño de nuestra jeringuilla resuelve el problema de la fatiga muscular gracias a su agarre de liberación suave. Esto directamente resuelve la insatisfacción. (PPAUSAA: 5 segundos) También quería saber qué otras áreas de su laboratorio se benefician de la reducción en el tiempo de procesamiento de muestras críticas. ¿Podría compartir eso conmigo?"},
    {"speaker": "cliente", "text": "El área de diagnóstico y el servicio de urgencias. Si las muestras tardan menos, los resultados llegan más rápido. Pero volviendo a la objeción: ¿me va a dar un rango de precios o no?"},
    {"speaker": "vendedor", "text": "Claro que sí. El rango de precios es de 7% a 10% superior al que usted maneja. Pero la reducción de errores al 1% le ahorra 300€ al mes en reprocesamiento. (PPAUSAA: 3 segundos) Y respecto al tiempo en urgencias, nuestra aguja reduce el tiempo en la camilla en un 20%. ¿Este doble ahorro justifica la diferencia de precio, señora García?"},
    {"speaker": "cliente", "text": "La justificación es buena, pero la diferencia de precio sigue siendo un problema de presupuesto trimestral para mí. Necesitaré ver los datos de compatibilidad con nuestro sistema de etiquetado automático y la garantía de suministro."},
    {"speaker": "vendedor", "text": "Excelente. La compatibilidad está garantizada al 100%. Y respecto al presupuesto, tenemos un modelo de suscripción que puede aliviar el golpe trimestral. (PPAUSAA: 4 segundos) Para avanzar, agendemos un encuentro de 20 minutos el próximo miércoles. Le enviaremos el informe técnico detallado sobre la compatibilidad y el modelo de suscripción. ¿Miércoles a las 3:00 pm le funciona para formalizar la prueba piloto?"},
    {"speaker": "cliente", "text": "El miércoles a las 3:00 pm no me funciona. Lo siento, tengo otra reunión. No me gusta que me fuerce la agenda. Y necesito el rango de precios final por escrito antes de agendar."},
    {"speaker": "vendedor", "text": "Disculpe, señora. (PPAUSAA: 4 segundos) Le enviaré el rango de precios por correo inmediatamente y el informe técnico. ¿Podríamos entonces agendar un follow-up el jueves a las 10:00 am?"},
    {"speaker": "cliente", "text": "Envíeme la información primero. Si todo está claro con el precio, le responderé para agendar. Gracias."}
]

transcript_8 = [
    {"speaker": "vendedor", "text": "Buenos días. Soy Darío de Soluciones Prime. He oído que su equipo médico valora mucho la eficiencia. Mi objetivo es que vean cómo podemos reducir sus errores de muestreo. ¿Cuál es el principal desafío que le impide alcanzar una tasa de error inferior al 1.5% actualmente?"},
    {"speaker": "cliente", "text": "Buenos días. El desafío es doble: la ergonomía del kit actual y, como siempre, el precio. Cualquier nuevo producto tiene que ser muy competitivo en costes para que lo consideremos."},
    {"speaker": "vendedor", "text": "Claro que sí. Me alegra mucho que mencione la ergonomía, porque ahí es donde nuestra innovación brilla. Nuestra jeringuilla tiene un sistema de doble flow que reduce la turbulencia. Esto ayuda. ¿Podría usted contarme un poco más sobre cómo esa frustración del personal se manifiesta en el día a día?"},
    {"speaker": "cliente", "text": "Se manifiesta en errores y en fatiga. Pero dígame, ¿por qué es tan importante para usted el doble flow y qué significa para mi presupuesto que sea un producto 'innovador'?"},
    {"speaker": "vendedor", "text": "Excelente pregunta! El doble flow es un término que usamos internamente para describir el diseño del émbolo, que minimiza el cizallamiento celular. Le aseguro que es un avance tecnológico. ¿Y qué otras áreas de su laboratorio se benefician si el proceso fuera más rápido?"},
    {"speaker": "cliente", "text": "Urgencias. Si las muestras llegan más rápido, el paciente es atendido antes. Pero, Darío, necesito el precio. ¿Está por debajo, a la par, o significativamente por encima de un proveedor de gama media?"},
    {"speaker": "vendedor", "text": "Comprendido. La reducción en el tiempo de procesamiento para urgencias es un valor inmenso. Y nuestra aguja es ultrafina. ¿Le parece más urgente resolver el problema del tiempo o el problema de la fatiga del personal?"},
    {"speaker": "cliente", "text": "Ambos. Pero el precio es la barrera inicial para mí. Necesito una cifra ahora mismo. Si no me da una banda de precios, no puedo avanzar."},
    {"speaker": "vendedor", "text": "Tiene toda la razón en que necesita la información financiera. Le enviaré un informe con el análisis de ROI detallado que demuestra el ahorro total en un trimestre. Agendemos un breve follow-up para discutir ese informe la próxima semana, ¿de acuerdo?"},
    {"speaker": "cliente", "text": "Un informe de ROI es útil, pero no es el precio. Además, ¿tienen la capacidad logística para abastecernos sin fallos? Nuestro proveedor actual tiene problemas de stock serios."},
    {"speaker": "vendedor", "text": "Claro que sí. Le confirmo que nuestra logística es robusta. De hecho, utilizamos un sistema de ERP de planificación que elimina los problemas de stock. ¿Le gustaría que le enviara también un folleto que detalla nuestras certificaciones de calidad ISO?"},
    {"speaker": "cliente", "text": "Sí, las certificaciones son útiles. Pero sobre el precio... ¿es la diferencia de precio algo que requeriría la aprobación de la Dirección General o solo la de su equipo?"},
    {"speaker": "vendedor", "text": "Entiendo la importancia de la Dirección General. Le prometo que el precio es competitivo y que el ROI lo justifica. Agendemos la reunión de 20 minutos el próximo miércoles. Le enviaremos toda la información (ROI, folleto, certificaciones) en ese email de agenda. ¿Miércoles a las 3:00 pm le funciona para formalizar el envío de la información?"},
    {"speaker": "cliente", "text": "No, Darío. No puedo agendar una reunión para recibir información que ya he solicitado tres veces. Envíeme el rango de precios en el cuerpo de un email. Si me funciona, le llamo. Gracias."}
]

transcript_9 = [
    {"speaker": "vendedor", "text": "Buenos días. Soy Lucía de Muestras Avanzadas. Mi objetivo es claro: reducir su tasa de error de muestreo y optimizar el costo de su personal. ¿Cuál es el principal desafío que le está impidiendo alcanzar una tasa de error inferior al 1.5% actualmente?"},
    {"speaker": "cliente", "text": "Buenos días. El desafío es la fatiga del personal; la ergonomía del kit es mala y el precio..."},
    {"speaker": "vendedor", "text": "Comprendido. La ergonomía. Eso es un dolor que resolvemos. Para cuantificarlo, ¿qué impacto financiero o de rotación tiene para usted esa insatisfacción del personal con el kit actual?"},
    {"speaker": "cliente", "text": "El costo de reemplazo de personal es alto, unos 2,000€ por flebotomista. Es nuestra mayor preocupación después del..."},
    {"speaker": "vendedor", "text": "Excelente dato. Entonces, el costo de la rotación es su dolor número uno. La jeringuilla tiene un diseño que elimina ese riesgo. ¿Podría usted compartir qué otros sistemas internos (etiquetado, clasificación) deben ser compatibles para que la solución funcione?"},
    {"speaker": "cliente", "text": "Debe ser compatible con nuestro sistema X-Lab para etiquetado automático, y, Lucía, necesito el precio. Me está preguntando por el dolor, pero no me da la solución financiera."},
    {"speaker": "vendedor", "text": "Claro que sí. El precio es un 10% superior. Pero, como usted dijo, la rotación cuesta 2,000€. Nuestra jeringuilla elimina esa causa de rotación y reduce su error del 3.5% al 1%."},
    {"speaker": "cliente", "text": "Esos son números buenos, pero ese 10% extra debe ser justificado ante el CFO. No puedo firmar sin su..."},
    {"speaker": "vendedor", "text": "Tiene toda la razón. El CFO es clave. Para él, vamos a enfocarnos en el ROI de la retención de personal. ¿Cuál es la fecha límite para implementar el nuevo sistema de trazabilidad que está esperando el CFO?"},
    {"speaker": "cliente", "text": "Finales del próximo trimestre. Si no se resuelve la fatiga y los errores, ese plazo es imposible de cumplir. Y me preocupa que el 10% de aumento genere resistencia en Logística."},
    {"speaker": "vendedor", "text": "Comprendido. Para resolver Logística: le garantizamos un SLA de suministro del 99.9%. Y para el CFO: agendemos una reunión de 20 minutos el próximo jueves. Le enviaremos un Business Case que muestra el ahorro en costos de reemplazo y el valor de completar la trazabilidad a tiempo. ¿Jueves a las 11:00 am le funciona?"},
    {"speaker": "cliente", "text": "El jueves tengo una reunión interna que se puede mover..."},
    {"speaker": "vendedor", "text": "Excelente. Mover la reunión. Le envío la invitación inmediatamente con la documentación adjunta. ¡Genial!"},
    {"speaker": "cliente", "text": "Espere, Lucía. No he dicho que la mueva. Envíe la documentación primero. Si el CFO la acepta, le llamo."}
]

transcript_10 = [
    {"speaker": "vendedor", "text": "Buenos días. Soy Jaime de Bio-Muestras. Eh, le llamo porque hemos visto que su laboratorio busca optimizar la eficiencia. Mira, mi objetivo es que vean cómo digamos que podemos reducir sus errores de muestreo. ¿Cuál es el principal desafío que le impide alcanzar una tasa de error inferior al 1.5% actualmente?"},
    {"speaker": "cliente", "text": "Buenos días. El desafío es la fatiga del personal; la ergonomía del kit es mala y, sobre todo, el precio. Cualquier nuevo producto que consideremos tiene que ser muy competitivo en costes."},
    {"speaker": "vendedor", "text": "Comprendido. O sea, entiendo que el coste y la ergonomía son las dos variables clave. A ver, hablemos de la ergonomía, que es donde nuestro diseño destaca. Digamos, que la fatiga del personal, ¿ha provocado un aumento en la rotación o en las bajas por estrés en el último año?"},
    {"speaker": "cliente", "text": "Sí. Hemos tenido un 15% de rotación en flebotomía. Es muy costoso. Pero, insisto: si son más caros que nuestro proveedor actual, es un obstáculo insalvable para mí."},
    {"speaker": "vendedor", "text": "Sí, lo entiendo. Bueno, le aseguro que nuestro diseño ergonómico reduce la fatiga. Esto directamente resuelve la insatisfacción que usted mencionó. Este, nuestro precio es un 8% superior, pero el ahorro por la reducción de errores al 1% compensa ese sobrecosto en tres meses. ¿Podría usted compartir conmigo qué otros sistemas internos (etiquetado, clasificación) deben ser compatibles para que la solución funcione en su día a día?"},
    {"speaker": "cliente", "text": "Debe ser compatible con nuestro sistema X-Lab para etiquetado automático. Y, Jaime, necesito la seguridad de que su empresa tiene la capacidad logística para abastecernos sin fallos. Nuestro proveedor actual tiene problemas de stock serios, esa es mi mayor preocupación."},
    {"speaker": "vendedor", "text": "Claro que sí. Mira, la compatibilidad con X-Lab está garantizada al 100%. Y respecto a la logística, la verdad es que garantizamos un SLA de suministro del 99.9%. O sea, eso se resuelve. Pero, volvamos al ROI que es lo que le interesa a su Dirección. ¿Cuál es la fecha límite para implementar el nuevo sistema de trazabilidad que está esperando el CFO?"},
    {"speaker": "cliente", "text": "Finales del próximo trimestre. Si no se resuelve la fatiga y los errores, ese plazo es imposible de cumplir. La trazabilidad es la prioridad número uno."},
    {"speaker": "vendedor", "text": "Fantástico. Este, eso es lo que quería saber. Le enviaré un Business Case que proyecta que nuestra jeringuilla le permitirá finalizar el proyecto de trazabilidad dos meses antes gracias al tiempo liberado. ¿Agendamos una reunión de 30 minutos el próximo jueves, e invitamos al CFO, para que usted pueda formalizar la prueba piloto?"},
    {"speaker": "cliente", "text": "El CFO está muy ocupado. Envíeme la justificación del ROI y la garantía de suministro por escrito. Si me parece convincente, le llamo para agendar la reunión con él."},
    {"speaker": "vendedor", "text": "Comprendido. A ver, para asegurarnos, ¿le envío el Business Case y la certificación de nuestro ERP logístico inmediatamente, y usted me da una hora para una llamada de seguimiento de 5 minutos el viernes solo para confirmar que recibió los documentos?"},
    {"speaker": "cliente", "text": "Envíe los documentos. No hace falta llamada de seguimiento. Si están bien, le contacto yo."}
]

transcripts = [transcript_1, transcript_2, transcript_3, transcript_4, transcript_5,
               transcript_6, transcript_7, transcript_8, transcript_9, transcript_10]

for transcript in transcripts:
    for d in transcript:
        d["duracion"] = 10

In [51]:
# Supongamos que tus transcripts están en variables transcript_1, transcript_2, ..., transcript_10
transcripts = [transcript_1, transcript_2, transcript_3, transcript_4, transcript_5,
               transcript_6, transcript_7, transcript_8, transcript_9, transcript_10]

# Diccionario para guardar resultados
resultados_scoring = {}

# Iterar sobre cada transcript
for i, transcript in enumerate(transcripts, start=1):
    score = evaluar_transcript_global(transcript)
    resultados_scoring[f"transcript_{i}"] = score

# Mostrar resultados
for k, v in resultados_scoring.items():
    print(f"{k}: {v}\n")


transcript_1: {'puntuacion_global': 85.2, 'detalle': {'muletillas_pausas': 100, 'claridad': 85, 'participacion': 60, 'cobertura': 100, 'preguntas': 100, 'ppm': 75}, 'pesos': {'muletillas_pausas': 0.15, 'claridad': 0.15, 'participacion': 0.25, 'cobertura': 0.15, 'preguntas': 0.2, 'ppm': 0.1}}

transcript_2: {'puntuacion_global': 88.5, 'detalle': {'muletillas_pausas': 100, 'claridad': 60, 'participacion': 100, 'cobertura': 70, 'preguntas': 100, 'ppm': 90}, 'pesos': {'muletillas_pausas': 0.15, 'claridad': 0.15, 'participacion': 0.25, 'cobertura': 0.15, 'preguntas': 0.2, 'ppm': 0.1}}

transcript_3: {'puntuacion_global': 93.2, 'detalle': {'muletillas_pausas': 100, 'claridad': 85, 'participacion': 95, 'cobertura': 85, 'preguntas': 100, 'ppm': 90}, 'pesos': {'muletillas_pausas': 0.15, 'claridad': 0.15, 'participacion': 0.25, 'cobertura': 0.15, 'preguntas': 0.2, 'ppm': 0.1}}

transcript_4: {'puntuacion_global': 85.8, 'detalle': {'muletillas_pausas': 100, 'claridad': 70, 'participacion': 80, 'c

In [40]:
calcular_muletillas(transcript=transcript_6)

{'puntuacion': 90,
 'penalizacion': 10,
 'total_muletillas': 3,
 'total_pausas': 1,
 'muletillas_usadas': ['eh', 'bueno', 'este']}

In [54]:
calcular_indice_preguntas(transcript_10)

{'puntuacion': 100,
 'total_preguntas': 6,
 'preguntas': ['¿cuál es el principal desafío que le impide alcanzar una tasa de error inferior al 1.5% actualmente?',
  '¿ha provocado un aumento en la rotación o en las bajas por estrés en el último año?',
  '¿podría usted compartir conmigo qué otros sistemas internos (etiquetado, clasificación) deben ser compatibles para que la solución funcione en su día a día?',
  '¿cuál es la fecha límite para implementar el nuevo sistema de trazabilidad que está esperando el cfo?',
  '¿agendamos una reunión de 30 minutos el próximo jueves, e invitamos al cfo, para que usted pueda formalizar la prueba piloto?',
  '¿le envío el business case y la certificación de nuestro erp logístico inmediatamente, y usted me da una hora para una llamada de seguimiento de 5 minutos el viernes solo para confirmar que recibió los documentos?'],
 'cerradas': 0,
 'sondeo': 0,
 'irrelevantes': 0,
 'penalizacion': 0,
 'bonificacion': 0}